In [4]:
file = r'd:/18-ds/github/Technical/Reading_different_format_files'

In [5]:
import json
json_file =file+'\intents.json'
with open(json_file,'r') as f:
    data = json.load(f)

In [6]:
import pandas as pd
df = pd.DataFrame(data)
df

,tag,patterns,responses
0,welcome,"[Hi, How are you, Is any one to talk?, Hello, ...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"[Bye, See you later, Goodbye, I will come back...","[See you later, thanks for visiting, have a gr..."
2,thankful,"[Thanks for helping me, Thank your guidance, T...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"[What hours are you open?, Tell your opening t...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"[Can I pay using credit card?, Can I pay usin...","[We accept VISA, Mastercard and credit card, W..."


In [7]:
df['patterns'] = df['patterns'].apply(', '.join) 

In [8]:
#conda install -c conda-forge textblob
!pip install textblob

In [ ]:
nltk.download('wordnet')

In [9]:
from nltk.corpus import stopwords
import string
from textblob import Word
stop = stopwords.words('english')
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x.lower() for x in x.split()))
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))

                                      
df['patterns'] = df['patterns'].str.replace('[^\w\s]','')
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x for x in x.split() if  not x.isdigit()))
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))

df['patterns'] = df['patterns'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df

,tag,patterns,responses
0,welcome,hi one talk hello hi available,"[Hello, thanks for contacting us, Good to see ..."
1,goodbye,bye see later goodbye come back soon,"[See you later, thanks for visiting, have a gr..."
2,thankful,thanks helping thank guidance thats helpful kind,"[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,hour open tell opening time open timing please,"[We're open every day 8am-7pm, Our office hour..."
4,payments,pay using credit card pay using mastercard pay...,"[We accept VISA, Mastercard and credit card, W..."


In [10]:
from gensim.models import Word2Vec

In [11]:
Bigger_list=[]
for i in df['patterns']:
    li = list(i.split(" "))
    Bigger_list.append(li)
Model= Word2Vec(Bigger_list,min_count=1,size=300,workers=4)

In [12]:
Model.save("word2vec.model")
Model.save("model.bin")

In [13]:
model = Word2Vec.load('model.bin')

In [14]:
vocab = list(model.wv.vocab)
vocab

['hi',
 'one',
 'talk',
 'hello',
 'available',
 'bye',
 'see',
 'later',
 'goodbye',
 'come',
 'back',
 'soon',
 'thanks',
 'helping',
 'thank',
 'guidance',
 'thats',
 'helpful',
 'kind',
 'hour',
 'open',
 'tell',
 'opening',
 'time',
 'timing',
 'please',
 'pay',
 'using',
 'credit',
 'card',
 'mastercard',
 'cash']

In [15]:
similar_words = model.most_similar('thanks')
print(similar_words)

[('open', 0.08885998278856277), ('pay', 0.08801586925983429), ('opening', 0.07690231502056122), ('goodbye', 0.07664275169372559), ('time', 0.0590304434299469), ('mastercard', 0.044141948223114014), ('come', 0.03937893360853195), ('bye', 0.031571608036756516), ('tell', 0.023389466106891632), ('hi', 0.018807880580425262)]


<ipython-input-15-c8e25aad01c6>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = model.most_similar('thanks')


In [16]:
dissimlar_words = model.doesnt_match('See you later, thanks for visiting'.split())
print(dissimlar_words)

thanks


<ipython-input-16-3453349c4aaf>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  dissimlar_words = model.doesnt_match('See you later, thanks for visiting'.split())
D:\Users\admin\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [17]:
similarity_two_words = model.similarity('please','see')
print("Please provide the similarity between these two words:")
print(similarity_two_words)

Please provide the similarity between these two words:
0.058276378


<ipython-input-17-27dad22e16f5>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  similarity_two_words = model.similarity('please','see')


In [18]:
similar = model.similar_by_word('kind')
print(similar)

[('goodbye', 0.10004822909832001), ('see', 0.08872447162866592), ('hour', 0.06222618371248245), ('thats', 0.05499551072716713), ('using', 0.0496162548661232), ('hi', 0.04873987287282944), ('please', 0.04473559185862541), ('talk', 0.030204754322767258), ('opening', 0.025168653577566147), ('mastercard', 0.01770235411822796)]


<ipython-input-18-9e21e225c8bc>:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  similar = model.similar_by_word('kind')
